In [1]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools
import random
import matplotlib.pyplot as plt
import timeit

In [2]:
#!pip3 install dwave-system

In [3]:
from dwave.system.composites import AutoEmbeddingComposite
from dwave.system.samplers import DWaveSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo

In [4]:
DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09" #"replace this string with actual token"

### 解析用データ作成

（3年間骨肉腫に罹患しない）事象と各特徴量とのP値を計算する。
- Osteogenic sarcoma data:骨肉腫
- LI:リンパ球浸出
- AOP:類骨病理学

In [5]:
import pandas as pd
import numpy as np

In [6]:
ost20 = pd.read_csv("../../input/ost20.csv", sep=',', index_col=0)
ost20.shape

(20, 4)

In [7]:
ost20.head()

,Y,LI,SEX,AOP
0,0,1,1,0
1,1,1,1,1
2,0,1,1,1
3,1,1,1,1
4,0,1,1,1


In [8]:
def make_t_list_columns_num_samples(df):
    t_list = []
    t_list.append(sum(df['Y']))
    t_list.append(np.dot(df['Y'], df['LI']))
    t_list.append(np.dot(df['Y'], df['SEX']))
    t_list.append(np.dot(df['Y'], df['AOP']))
    
    columns = list(df.columns)
    num_samples = df.shape[0]
    
    return t_list, columns, num_samples 

In [9]:
t_list, columns, num_samples = make_t_list_columns_num_samples(ost20)

### annealing_time = 20, num_reads=100, for t1 in range(0, ost16.shape[0]+1)

In [10]:
dw_sampler = DWaveSampler(
    endpoint="https://cloud.dwavesys.com/sapi",
    solver = 'DW_2000Q_6',
    token = DWAVE_TOKEN
)

In [11]:
qa_sampler = AutoEmbeddingComposite(dw_sampler)

In [12]:
def find_valid_y(df, num_reads):
    valid_y_list = {}
    valid_y_num = {}
    for t1 in range(0, df.shape[0]+1):
        y = Array.create('y', shape=df.shape[0], vartype='BINARY')

        #QUBO式で定式化
        H = (sum(y) - t_list[0])**2
        H += (np.dot(df[columns[1]], y) - t1)**2
        for j in range(2, len(columns)):
            H_plus = (np.dot(df[columns[j]], y) - t_list[j])**2
            H += H_plus

        qb = H.compile().to_qubo()
        res = qa_sampler.sample_qubo(qb[0], num_reads=num_reads)

        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        for y_info in list(res.record):
            if sum(y_info[0]) == t_list[0]:#int同士の比較
                y = pd.Series(y_info[0])
                if np.dot(df[columns[1]], y) == t1:#int同士の比較
                    for j in range(2, len(columns)):
                        if np.dot(df[columns[j]], y) != t_list[j]:#int同士の比較
                            break
                    else:
                        if all(list(y_info[0]) != p for p in valid_y_list[t1]):
                            valid_y_num[t1] += 1
                            valid_y_list[t1].append(list(y_info[0]))
                            print('perfect')
    return valid_y_list, valid_y_num

In [13]:
valid_y_list, valid_y_num = find_valid_y(ost20, 100)
print(valid_y_list)
print('--------')
print(valid_y_num)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: []}
--------
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0}


In [14]:
loop = 10
df = ost20
num_reads = 100
result = timeit.timeit('find_valid_y(df, num_reads)', globals=globals(), number=loop)
print(result / loop, '秒')

perfect
perfect
perfect
perfect
perfect
perfect
121.85455411000002 秒


In [15]:
valid_y_list

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: []}

In [16]:
valid_y_num

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0}

In [17]:
valid_y_num[9]/sum(valid_y_num.values())

ZeroDivisionError: division by zero